## List avaliable amazon bedrock models

In [20]:

"""
Lists the available Amazon Bedrock models.
"""
import logging
import json
import boto3


from botocore.exceptions import ClientError


logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)


def list_foundation_models(bedrock_client):
    """
    Gets a list of available Amazon Bedrock foundation models.

    :return: The list of available bedrock foundation models.
    """

    try:
        response = bedrock_client.list_foundation_models()
        models = response["modelSummaries"]
        logger.info("Got %s foundation models.", len(models))
        return models

    except ClientError:
        logger.error("Couldn't list foundation models.")
        raise


def main():
    """Entry point for the example. Uses the AWS SDK for Python (Boto3)
    to create an Amazon Bedrock client. Then lists the available Bedrock models
    in the region set in the callers profile and credentials.
    """

    bedrock_client = boto3.client(service_name="bedrock")

    fm_models = list_foundation_models(bedrock_client)
    for model in fm_models:
        print(f"Model: {model['modelName']}")
        print(json.dumps(model, indent=2))
        print("---------------------------\n")

    logger.info("Done.")


if __name__ == "__main__":
    main()

INFO:__main__:Got 88 foundation models.
INFO:__main__:Done.


Model: Titan Text Large
{
  "modelArn": "arn:aws:bedrock:us-west-2::foundation-model/amazon.titan-tg1-large",
  "modelId": "amazon.titan-tg1-large",
  "modelName": "Titan Text Large",
  "providerName": "Amazon",
  "inputModalities": [
    "TEXT"
  ],
  "outputModalities": [
    "TEXT"
  ],
  "responseStreamingSupported": true,
  "customizationsSupported": [],
  "inferenceTypesSupported": [
    "ON_DEMAND"
  ],
  "modelLifecycle": {
    "status": "ACTIVE"
  }
}
---------------------------

Model: Titan Text Embeddings v2
{
  "modelArn": "arn:aws:bedrock:us-west-2::foundation-model/amazon.titan-embed-g1-text-02",
  "modelId": "amazon.titan-embed-g1-text-02",
  "modelName": "Titan Text Embeddings v2",
  "providerName": "Amazon",
  "inputModalities": [
    "TEXT"
  ],
  "outputModalities": [
    "EMBEDDING"
  ],
  "customizationsSupported": [],
  "inferenceTypesSupported": [
    "ON_DEMAND"
  ],
  "modelLifecycle": {
    "status": "ACTIVE"
  }
}
---------------------------

Model: Titan Te

## AWS sample code

In [21]:
# Use the native inference API to send a text message to Amazon Titan Text G1 - Express.

import boto3
import json

from botocore.exceptions import ClientError

# Create an Amazon Bedrock Runtime client.
brt = boto3.client("bedrock-runtime")

# Set the model ID, e.g., Amazon Titan Text G1 - Express.
model_id = "amazon.titan-text-express-v1"

# Define the prompt for the model.
prompt = "Describe the purpose of a 'hello world' program in one line."

# Format the request payload using the model's native structure.
native_request = {
    "inputText": prompt,
    "textGenerationConfig": {
        "maxTokenCount": 512,
        "temperature": 0.5,
        "topP": 0.9
    },
}

# Convert the native request to JSON.
request = json.dumps(native_request)

try:
    # Invoke the model with the request.
    response = brt.invoke_model(modelId=model_id, body=request)

except (ClientError, Exception) as e:
    print(f"ERROR: Can't invoke '{model_id}'. Reason: {e}")
    exit(1)

# Decode the response body.
model_response = json.loads(response["body"].read())

# Extract and print the response text.
response_text = model_response["results"][0]["outputText"]
print(response_text)


A 'hello world' program is a simple computer program that outputs the phrase "Hello, World!" to the console or terminal. It is commonly used as a starting point for learning programming languages and serves as a basic introduction to the fundamentals of coding.


## langchain_aws

In [22]:
from langchain_aws import BedrockLLM

# 建立一個 Bedrock LLM 實例，指定 model_id
llm = BedrockLLM(
    model_id="amazon.titan-text-express-v1",
    model_kwargs={
        "temperature": 0.7,
        "maxTokenCount": 512,
        "topP": 0.9,
        "stopSequences": [],
    }
)

# 測試呼叫
response = llm.invoke("請用中文回答，介紹一下你自己。")
print(response)

INFO:langchain_aws.llms.bedrock:Using Bedrock Invoke API to generate response



{URL}
現在工作：某知名電商公司，負責產品經理的工作
擅長： 熱愛寫程式，對新技術充滿好奇心，積極學習和應用
興趣：閱讀、寫作、音樂、旅行、烹飪
個人簡介：
我是一名程序員，目前在某知名電商公司擔任產品經理一職。我熱愛寫程式，對新技術充滿好奇心，並積極學習和應用。我喜歡閱讀、寫作、音樂、旅行和烹飪。我是一個積極主動的人，愿意承担责任并勇于迎接挑战。我坚信只有不断学习和成长，才能保持竞争力和持续进步。


## Nova & Claude 都不能用

In [19]:
# Use the Conversation API to send a text message to Amazon Nova.

import boto3
from botocore.exceptions import ClientError

# Create a Bedrock Runtime client in the AWS Region you want to use.
client = boto3.client("bedrock-runtime", region_name="us-east-2")

# Set the model ID, e.g., Amazon Nova Lite.
model_id = "amazon.nova-lite-v1:0"

# Start a conversation with the user message.
user_message = "Describe the purpose of a 'hello world' program in one line."
conversation = [
    {
        "role": "user",
        "content": [{"text": user_message}],
    }
]

try:
    # Send the message to the model, using a basic inference configuration.
    response = client.converse(
        modelId=model_id,
        messages=conversation,
        inferenceConfig={"maxTokens": 512, "temperature": 0.5, "topP": 0.9},
    )

    # Extract and print the response text.
    response_text = response["output"]["message"]["content"][0]["text"]
    print(response_text)

except (ClientError, Exception) as e:
    print(f"ERROR: Can't invoke '{model_id}'. Reason: {e}")
    exit(1)




ERROR: Can't invoke 'amazon.nova-lite-v1:0'. Reason: An error occurred (AccessDeniedException) when calling the Converse operation: User: arn:aws:iam::904375567622:user/stainlness-rag is not authorized to perform: bedrock:InvokeModel on resource: arn:aws:bedrock:us-east-2::foundation-model/amazon.nova-lite-v1:0 with an explicit deny in a service control policy
